In [1]:
class Symbol:
  def __init__(self, val, index):
    self.val = val
    self.index = index

class StateMachine:
  def __init__(self, pattern):
    self.pattern = pattern
    self.pi = self.build(pattern)

  def build_state_cache(self, fsst_symbols):
    assert self.pattern is not None

    self.fsst_symbols = fsst_symbols
    self.cache_state = [-1] * (len(self.pattern) * len(fsst_symbols))

    for i in range(len(self.pattern)):
      for symbol in fsst_symbols:
        # Save the state.
        tmp = self.curr_state

        # Set the current state to position `i`.
        self.curr_state = i

        # Simulate.
        self.accept_symbol(symbol)

        # Cache the state we arrived at.
        self.cache_state[i * len(fsst_symbols) + symbol.index] = self.curr_state

        # And reset.
        self.curr_state = tmp
        
  def init(self):
    self.curr_state = 0

  def accept_letter(self, letter):
    while (self.curr_state > 0) and (self.pattern[self.curr_state] != letter):
      self.curr_state = self.pi[self.curr_state - 1]
    
    if self.pattern[self.curr_state] == letter:
      self.curr_state += 1

  def accept_symbol(self, symbol):
    for letter in symbol.val:
      self.accept_letter(letter)

      # Already reached the final state?
      if self.curr_state == len(self.pattern):
        return

  def accept_cached_symbol(self, symbol):
    assert self.fsst_symbols is not None
    return self.cache_state[self.curr_state * len(self.fsst_symbols) + symbol.index]

  @staticmethod
  def build_pi(self, P):
    # Init.
    pi = [0] * len(P)

    # Nowhere to go.
    pi[0] = 0

    # Init the state.
    k = 0
    for q in range(1, len(P)):
      while (k > 0) and (P[k] != P[q]):
        k = pi[k - 1]
      
      # Match? Then advance.
      if P[k] == P[q]:
        k += 1
      
      # Store the state.
      pi[q] = k

    return pi
  
  def naive_match(self, text):
    # Init.
    self.init()

    for i in range(len(text)):
      if isinstance(text[i], Symbol):
        self.accept_symbol(text[i])
      else:
        self.accept_letter(text[i])
      
      if self.curr_state == len(self.pattern):
        return True
      
    return False
  
  def cached_match(self, text):
    # Init.
    self.init()

    for i in range(len(text)):
      if isinstance(text[i], Symbol):
        self.accept_cached_symbol(text[i])
      else:
        self.accept_letter(text[i])
      
      if self.curr_state == len(self.pattern):
        return True
      
    return False


_IncompleteInputError: incomplete input (884067878.py, line 2)